In [1]:
%pwd

'c:\\Users\\nguye\\Documents\\GitHub\\StressDetector-EEG'

In [2]:
from torcheeg.datasets import DEAPDataset
from torcheeg import transforms
import torch
from src.utils.transforms import DeapAVToStress

from torcheeg.datasets.constants.emotion_recognition.deap import \
    DEAP_CHANNEL_LOCATION_DICT

dataset = DEAPDataset(
    io_path=f'.data_cache/deap',
    root_path='.data/DEAP/data_preprocessed_python-002/data_preprocessed_python',
    online_transform=transforms.Compose([
        transforms.To2d(),
        transforms.ToTensor()
    ]),
    label_transform=transforms.Compose([
        transforms.Select(['arousal','valence']),
        # transforms.Binary(5.0),
        # transforms.BinariesToCategory(),
        DeapAVToStress(thresholds=[
            [7.5, 2.5],
            [5.0, 5.0]]),
    ]),
    num_worker=8)

[2025-08-14 17:40:58] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from .data_cache/deap.


In [3]:
train_val_test_split = [0.6, 0.2, 0.2]

train_size = int(train_val_test_split[0] * len(dataset))
val_size = int(train_val_test_split[1] * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

In [4]:
from torch.utils.data import DataLoader
batch_size = 64

train_loader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 2,
    pin_memory = True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle = False,
    num_workers = 2,
    pin_memory = True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle = False,
    num_workers = 2,
    pin_memory = True,
)

In [5]:
from torch.optim import SGD, Adam
from mmengine.runner import Runner
from src.models.torcheeg_mmwraper import MMEEGNet
from src.utils.metrics import AccuracyWithLoss, Accuracy, ConfusionMatrix


runner = Runner(
    # the model used for training and validation.
    # Needs to meet specific interface requirements

    model=MMEEGNet(chunk_size=128,
                num_electrodes=32,
                dropout=0.5,
                kernel_1=64,
                kernel_2=16,
                F1=8,
                F2=16,
                D=2,
                num_classes=3).float(),

    # working directory which saves training logs and weight files
    work_dir='./.exp/new_deap_stress/eegnet',
    
    # train dataloader needs to meet the PyTorch data loader protocol
    train_dataloader=train_loader,
    # optimize wrapper for optimization with additional features like
    # AMP, gradtient accumulation, etc
    optim_wrapper=dict(optimizer=dict(type=Adam, 
                                      lr=0.001,
                                      betas=(0.9, 0.999),
                                      eps=1e-08)),
    # trainging coinfs for specifying training epoches, verification intervals, etc
    train_cfg=dict(by_epoch=True, 
                   max_epochs=200, 
                   val_interval=1),
    
    
    # validation dataloader also needs to meet the PyTorch data loader protocol
    val_dataloader=val_loader,
    # validation configs for specifying additional parameters required for validation
    val_cfg=dict(),
    # validation evaluator. The default one is used here
    val_evaluator=dict(type=AccuracyWithLoss),

    # test dataloader also needs to meet the PyTorch data loader protocol
    test_dataloader=test_loader,
    # test configs for specifying additional parameters required for testing
    test_cfg=dict(),
    # test evaluator. The default one is used here
    test_evaluator=[dict(type=Accuracy), dict(type=ConfusionMatrix, num_classes=3)],

    visualizer=dict(type='Visualizer', vis_backends=[dict(type='TensorboardVisBackend')]),
    
    resume=True,
)

08/14 17:41:04 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
    CUDA available: False
    MUSA available: False
    numpy_random_seed: 167233273
    MSVC: n/a, reason: fileno
    PyTorch: 2.8.0+cpu
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 193833145
  - Intel(R) oneAPI Math Kernel Library Version 2025.2-Product Build 20250620 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.7.1 (Git Hash 8d263e693366ef8db40acc569cc7d8edf644556d)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, COMMIT_SHA=a1cb3cc05d46d198467bebbb6e8fba50a325d4e7, CXX_COMPILER=C:/actions-runner/_work/pytorch/pytorch/pytorch/.ci/pytorch/windows/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WIN

In [ ]:
runner.train()

08/14 17:41:04 - mmengine - WARNING - Dataset Subset has no metainfo. ``dataset_meta`` in visualizer will be None.
08/14 17:41:04 - mmengine - WARNING - The prefix is not set in metric class AccuracyWithLoss.
08/14 17:41:04 - mmengine - WARNING - Dataset Subset has no metainfo. ``dataset_meta`` in evaluator, metric and visualizer will be None.
Did not find last_checkpoint to be resumed.
08/14 17:41:04 - mmengine - INFO - Auto resumed from the latest checkpoint None.
08/14 17:41:04 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
08/14 17:41:04 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
08/14 17:41:04 - mmengine - INFO - Checkpoints will be saved to c:\Users\nguye\Documents\GitHub\StressDetector-EEG\.exp\new_deap_stress\eegnet.


c:\Users\nguye\Documents\GitHub\StressDetector-EEG\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


08/14 17:41:11 - mmengine - INFO - Epoch(train)   [1][ 10/720]  lr: 1.0000e-03  eta: 1 day, 3:04:42  time: 0.6770  data_time: 0.6534  loss: 1.0874
08/14 17:41:11 - mmengine - INFO - Epoch(train)   [1][ 20/720]  lr: 1.0000e-03  eta: 14:11:42  time: 0.0328  data_time: 0.0113  loss: 1.0687
08/14 17:41:11 - mmengine - INFO - Epoch(train)   [1][ 30/720]  lr: 1.0000e-03  eta: 9:44:49  time: 0.0213  data_time: 0.0002  loss: 1.0572
08/14 17:41:11 - mmengine - INFO - Epoch(train)   [1][ 40/720]  lr: 1.0000e-03  eta: 7:31:25  time: 0.0214  data_time: 0.0013  loss: 1.0253
08/14 17:41:12 - mmengine - INFO - Epoch(train)   [1][ 50/720]  lr: 1.0000e-03  eta: 6:11:56  time: 0.0225  data_time: 0.0004  loss: 1.0010
08/14 17:41:12 - mmengine - INFO - Epoch(train)   [1][ 60/720]  lr: 1.0000e-03  eta: 5:18:32  time: 0.0215  data_time: 0.0004  loss: 0.9714
08/14 17:41:12 - mmengine - INFO - Epoch(train)   [1][ 70/720]  lr: 1.0000e-03  eta: 4:40:19  time: 0.0213  data_time: 0.0002  loss: 0.9461
08/14 17:41:

In [ ]:
# Run trained model on test set
test_results = runner.test()

In [ ]:
test_results

In [ ]:
ConfusionMatrix().plot(test_results['confusion_matrix/result'], classes=['No Stress', 'Slight Stress', 'High Stress'], include_values=True, normalize=True)